# tic tac toe is intended for two players, in that occasion for the human player, and a computer player. 

In [1]:
# Importing libraries - numpy for representing the board numerically and vector operations,  
# random for making choices regarding who starts etc.
import random
import numpy as np

In [2]:
"""
Board class stores the gameboard's state. It includes methods that allow to show
and update the board, find the free spaces, and sum the rows/cols/diagonals.
The game's algorithm and other functionalities allow only 3 x 3 board space.

"""
class Board: 
    
    def __init__(self, game):
        # reference to Game class' object
        self.game = game
        # constants with desired number of rows/columns
        self.board_rows = 3
        self.board_cols = 3
        # The player's symbol is coded with 1, while game's symbol is coded with -1.
        self.board = np.zeros((self.board_rows, self.board_cols))
        self.possible_coordinates = self.get_possible_coordinates() # the list of gameboard's spaces.
        
    """
    Prints the gameboard. The board is initially filled with zeros, coded to display as empty spaces.

    """
    def show_board(self): 
        for row in range(0, self.board_rows):
            print('------------')
            out = '| '
            for col in range(0, self.board_cols):
                if self.board[row, col] == 1:
                    token = self.game.players[0].symbol
                elif self.board[row, col] == -1:
                    token = self.game.players[1].symbol
                elif self.board[row, col] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('------------')

    """
    Returns the list of available spaces' coordinates by checking if they contain zeros.
    """ 
    def get_possible_coordinates(self):
        possible_coordinates_list = []
        for row in range(0, self.board_rows):
            for column in range(0, self.board_cols):
                if self.board[row, column] == 0:
                    possible_coordinates_list.append((row, column))
        return possible_coordinates_list
    
    """
    Updates the board depending on who made a move (with either 1 or -1). 
    It takes the 'coordinates' argument, a tuple that stores input coordinates.
    """

    def update_board(self, coordinates):
        self.board[coordinates] = self.game.current_player_num
        self.possible_coordinates = self.get_possible_coordinates()
        
    """ 
    Since the symbols (X, empty spaces, and O) are coded numbers (-1, 0, and 1), 
    In order to determine the winner, I decided to check the sums of my board's rows, columns, and diagonals. 
    The function returns the list of the board's rows, columns, and diagonals' sums. (8 elements in total)
    """ 
    def get_sums(self):
        row_sums = self.board.sum(axis = 1) 
        col_sums = self.board.sum(axis = 0) 
        diag1_sum = self.board.trace() 
        diag2_sum = np.fliplr(self.board).trace() #sum of the second diagonal - since the board is a square, we can use numpy's fliplr()
        sums_list = [] #empty list to store all the sums. 
        for array in [row_sums, col_sums]:
            for sum in array:
                sums_list.append(sum) 
        for sum in diag1_sum, diag2_sum:
            sums_list.append(sum)
        return sums_list

In [3]:
"""
The class HumanPlayer and its' child class, ComputerPlayer, store attribute symbol:
the symbol (X or O) they are represented with on the gameboard, 
and the method which allows them to choose their moves as instructed
(Inputs for HumanPlayer and random choices for ComputerPlayer)
"""

# human player can choose their move
class HumanPlayer:
    
    def __init__(self, symbol):
        self.symbol = symbol # the player's symbol (X or O)
    
    """
    The function choose_move from the Player's class asks the human player to input their desired coordinates and store them. 
    It returns the tuple with coordinates.
    """ 
    def choose_move(self, possible_coordinates_list):
        while True:
            row = int(input("Enter the row (1 - 3): "))
            col = int(input("Enter the col (1 - 3): "))
            if (row not in range(1, 4)) or (col not in range(1, 4)): # if the move is outside of the board, the game will prompt
                print('That space is not on a board.')               # the player to pick a valid move. 
                continue
            coordinates = (row - 1, col - 1)                          # numpy's arrays are indexed from 0, which might be counter-intuitive. 
            if (coordinates not in possible_coordinates_list):         # you can't overwrite computer's move.
                print("Illegal move!")                               # the player will be prompted to input a valid one.              
                continue
            else:
                return coordinates
            
"""
The computer decided on the moves randomly by picking one from the list of free spaces.        
"""
class ComputerPlayer(HumanPlayer):
    
    def choose_move(self, possible_coordinates_list):
        coordinates = random.choice(possible_coordinates_list)
        return coordinates


In [4]:
"""
Class Game stores the methods related to tic tac toe's logic and the game's dynamic.
""" 

class Game:
    
    """
    The function prompts the human player to choose whether they prefer X or O 
    until they provide a valid input. You can use upper or lower case. 
    It returns the list of symbols: the symbol the human player is playing with 
    and the remaining symbol that will be used by the computer.

    """
    def choose_symbols():
        symbols = ['X','O'] # possible symbols
        while True:
            print("Do you prefer X or O?") 
            symbol = input("Your choice: ").upper()
            if symbol in symbols:
                symbols.remove(symbol)
                other_symbol = symbols[0]
                print("Your symbol is: ", symbol)
                print("Computer's symbol is: ", other_symbol)
                return [symbol, other_symbol]
            else:
                print("Sorry, you can play only with O and X.")
                
    
    user_num = 1 # assumption: user_num is always 1.
    comp_num = -1 # assumption: comp_num is always -1.
    
    def __init__(self, player1, player2):
        self.players = [player1, player2] # the list of the players' objects (human player and computer)
    
    """
    The function randomly decides whether the human player or computer player goes first.
    Return an int of either -1 or 1.
    """
    def who_starts(self):
        start_num = random.choice([1, -1])
        if start_num == self.user_num:
            print("You can start!")
        else:
            print("Computer will start")
        return start_num
    
    """
    The function returns a current player's object. 
    """
    def get_current_player(self):
        if self.current_player_num == 1:
            return self.players[0]
        else:
            return self.players[1]
        
    """
    The function switches between players so that they can take their turns. Communicates whose turn it is.
    """
    def switch_current_player(self):
        print()
        if self.current_player_num == 1:
            self.current_player_num = -1
            print("Computer's turn")
        else:
            self.current_player_num = 1
            print("Your turn!")
    
    """
    Since a player needs at least three moves to win, and the other player has to be after the second turn as well by this point,
    there is no point in checking if the game is finished before the number of empty spaces on the board is bigger than 4.
    Since a computer inputs only -1, and human inputs only 1, the computed sums can tell whether someone has already won.
    In the case of a draw, the game continues until the board is full. 
    The function returns a boolean variable: 
    True if the game is over, 
    False if the winner or a tie hasn't been determined yet.
    Implemented for the sake of efficiency.
    """
    def is_gameover(self):
        if len(self.board.possible_coordinates) < 5:
            sums_list = self.board.get_sums()
            if 3 in sums_list:
                print("You won!")
                return True
            elif -3 in sums_list:
                print("You lost.")
                return True
            elif len(self.board.possible_coordinates) == 0:
                print("Board full, it's a tie.")
                return True
        return False
   
    """The function play is responsible for the game logic and allows the player to play."""
    def play(self):
        gameover = False # gameover is a boolean - if it's False, it means that the game is on.
        self.board = Board(self) # the board the player can play on.
        self.current_player_num = self.who_starts() 
        self.board.show_board() 
        while not gameover: #if the game is on:
            current_player = self.get_current_player() # check whether the move is player's or computer's
            coordinates = current_player.choose_move(self.board.possible_coordinates) # save's current player's choice
            self.board.update_board(coordinates) # updating the board with the desired coordinates.
            self.board.show_board() 
            if self.is_gameover(): # if the conditions from is_gameover() are met, start checking whether the game finished.
                gameover = True
            else:
                self.switch_current_player() # if the game is still on, the players should switch.

    """
     If the player inputs 'y', 'Y,' or any word starting with 'y', start the game again.
    Returns a boolean variable (True/False) indicating whether the human player wishes to play more or not.
    """
    def play_again(self):
        replay = input('Do you want to play again? (Y/N): ')
        if replay.upper().startswith('Y'):
            self.play()
            return True
        else:
            return False

In [5]:
if __name__ == "__main__":
    #choose the preferred symbol
    symbols = Game.choose_symbols()
    # create Player objects
    player = HumanPlayer(symbols[0])
    computer = ComputerPlayer(symbols[1])
    # create Game between player and computer
    game = Game(player, computer)
    # start game
    game.play()
    # you can play as much as you wish:
    while game.play_again(): continue

Do you prefer X or O?
Your choice: x
Your symbol is:  X
Computer's symbol is:  O
Computer will start
------------
|   |   |   | 
------------
|   |   |   | 
------------
|   |   |   | 
------------
------------
|   |   | O | 
------------
|   |   |   | 
------------
|   |   |   | 
------------

Your turn!
Enter the row (1 - 3): 2
Enter the col (1 - 3): 2
------------
|   |   | O | 
------------
|   | X |   | 
------------
|   |   |   | 
------------

Computer's turn
------------
|   |   | O | 
------------
|   | X |   | 
------------
|   | O |   | 
------------

Your turn!
Enter the row (1 - 3): 1
Enter the col (1 - 3): 1
------------
| X |   | O | 
------------
|   | X |   | 
------------
|   | O |   | 
------------

Computer's turn
------------
| X |   | O | 
------------
|   | X |   | 
------------
| O | O |   | 
------------

Your turn!
Enter the row (1 - 3): 3
Enter the col (1 - 3): 3
------------
| X |   | O | 
------------
|   | X |   | 
------------
| O | O | X | 
------------
